In [1]:
import os
import glob

import numpy as np
import scikit_posthocs as sp # pip3 install scikit-posthocs
import scipy.stats as ss
import pandas as pd
from autorank import autorank, create_report, plot_stats # pip3 install autorank
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import gridspec

In [2]:
def read_baard_results(data, model, attacks):
    list_df = []
    for i in range(5):
        for a in attacks:
            path = os.path.join(
                '..','results', 'result_{}'.format(i), '{}_{}'.format(data, model), 'results', 
                '{}_{}_{}_baard.csv'.format(data, model, a))
            df = pd.read_csv(path)
            df = df.drop('Unnamed: 0', axis=1)
            list_df.append(df)
    df = pd.concat(list_df, axis=0, ignore_index=True)
    df = df.rename(columns={
        'acc_on_adv_3': 'acc_on_adv',
        'fpr_3': 'fpr'
    })
    df['defence'] = 'baard'
    return df[['data', 'model', 'attack', 'adv_param', 'defence', 'acc_no_def', 'acc_on_adv', 'fpr']]

In [3]:
def merge_results(data, model, attacks, defences):
    list_df = []
    for i in range(5):
        for a in attacks:
            for d in defences:
                p_mnist_dnn_apgd2_baard = os.path.join(
                    '..', 'results', 'result_{}'.format(i), '{}_{}'.format(data, model), 'results', 
                    '{}_{}_{}_{}.csv'.format(data, model, a, d))
                df = pd.read_csv(p_mnist_dnn_apgd2_baard)
                df = df.drop('Unnamed: 0', axis=1)
                df['defence'] = d
                list_df.append(df)
    df = pd.concat(list_df, axis=0, ignore_index=True)
    return df

In [4]:
# Load MNIST
attacks = ['fgsm', 'apgd', 'apgd2', 'cw2', 'deepfool', 'line']

df = read_baard_results('mnist', 'dnn', attacks)

defences = ['fs', 'lid', 'magnet', 'rc']
df = df.append(merge_results('mnist', 'dnn', attacks, defences))

# Load CIFAR10 ResNet
df = df.append(read_baard_results('cifar10', 'resnet', attacks))
df = df.append(merge_results('cifar10', 'resnet', attacks, defences))

# Load CIFAR10 Vgg
df = df.append(read_baard_results('cifar10', 'vgg', attacks))
df = df.append(merge_results('cifar10', 'vgg', attacks, defences))

In [5]:
# Numeric DNN
attacks = ['fgsm', 'apgd', 'apgd2', 'cw2']
defences = ['lid', 'magnet', 'rc']

data_num = ['banknote', 'breastcancer', 'htru2']
for d in data_num: 
    df = df.append(read_baard_results(d, 'dnn', attacks))
    df = df.append(merge_results(d, 'dnn', attacks, defences))

In [6]:
# SVM
attacks = ['fgsm', 'bim', 'boundary']
defences = ['magnet', 'rc']

for d in data_num:
    df = df.append(read_baard_results(d, 'svm', attacks))
    df = df.append(merge_results(d, 'svm', attacks, defences))

In [7]:
# Tree
attacks = ['tree', 'boundary']
defences = ['magnet', 'rc']

for d in data_num:
    df = df.append(read_baard_results(d, 'tree', attacks))
    df = df.append(merge_results(d, 'tree', attacks, defences))

In [8]:
df = df.sort_values(by=['data', 'model', 'attack', 'adv_param', 'defence'], ascending=True)
df

,data,model,attack,adv_param,defence,acc_no_def,acc_on_adv,fpr
7,banknote,dnn,apgd,0.05,baard,0.688442,0.924623,0.000000
30,banknote,dnn,apgd,0.05,baard,0.728643,0.939698,0.020101
53,banknote,dnn,apgd,0.05,baard,0.722222,0.904040,0.005051
76,banknote,dnn,apgd,0.05,baard,0.761421,0.923858,0.010152
99,banknote,dnn,apgd,0.05,baard,0.761421,0.923858,0.010152
...,...,...,...,...,...,...,...,...
99,mnist,dnn,line,1.00,rc,0.970000,0.970000,0.000000
199,mnist,dnn,line,1.00,rc,0.977000,0.977000,0.000000
299,mnist,dnn,line,1.00,rc,0.967000,0.966000,0.001000
399,mnist,dnn,line,1.00,rc,0.971000,0.963000,0.002000


In [9]:
df.to_csv('full_results.csv', index=False)